In [12]:
pip install --upgrade pip

^C
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pygeohash

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark ETL Task") \
    .getOrCreate()

df_restaurant = spark.read.csv('data/restaurant/part-00000-c8acc470-919e-4ea9-b274-11488238c85e-c000.csv', header=True, inferSchema=True)
# df_weather = spark.read.csv('data/geo/8/day=01/.part-00037-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet.crc', header=True, inferSchema=True)
df_restaurant

DataFrame[id: bigint, franchise_id: int, franchise_name: string, restaurant_franchise_id: int, country: string, city: string, lat: double, lng: double]

In [31]:
import os
from pyspark.sql.types import StructType

def read_data(spark, path, file_type):
    """
    Reads all files of the specified type from all subdirectories within the provided path 
    and concatenates them into a single DataFrame. It recursively searches every subdirectory,
    no matter the depth, for files ending in '.snappy.parquet' if 'parquet' is specified as file_type.
    
    Args:
        spark (SparkSession): The Spark session instance.
        path (str): The root directory path that contains the subdirectories with the files.
        file_type (str): The type of files to read ('csv' or 'parquet').
        
    Returns:
        DataFrame: A DataFrame containing all data from the files found with the specified extension.
        
    Raises:
        ValueError: If an unsupported file type is specified or no files are found.
    """
    if file_type not in ['csv', 'parquet']:
        raise ValueError("Unsupported file type. Please use 'csv' or 'parquet'.")

    all_files = []
    # Walk through directory
    for subdir, dirs, files in os.walk(path):
        for file in files:
            # Check for file extension
            if file_type == 'csv' and file.endswith(".csv"):
                all_files.append(os.path.join(subdir, file))
            elif file_type == 'parquet' and file.endswith(".snappy.parquet"):
                all_files.append(os.path.join(subdir, file))

    # Read all files into a DataFrame
    if all_files:
        if file_type == 'csv':
            df = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load(all_files)
        elif file_type == 'parquet':
            df = spark.read.parquet(all_files)
    else:
        # Provide a schema or handle the empty dataset case differently
        empty_schema = StructType([])  # Define an empty schema
        df = spark.createDataFrame([], schema=empty_schema)  # Create an empty DataFrame with a defined schema

    return df

In [14]:
# Read and concatenate data
dataframe = read_data(spark, 'data/restaurant/', 'csv')

# Show the first few rows of the DataFrame
dataframe.show()

+------------+------------+--------------------+-----------------------+-------+--------------+------+-------+
|          id|franchise_id|      franchise_name|restaurant_franchise_id|country|          city|   lat|    lng|
+------------+------------+--------------------+-----------------------+-------+--------------+------+-------+
|197568495625|          10|    The Golden Spoon|                  24784|     US|       Decatur|34.578|-87.021|
| 17179869242|          59|         Azalea Cafe|                  10902|     FR|         Paris|48.861|  2.368|
|214748364826|          27|     The Corner Cafe|                  92040|     US|    Rapid City| 44.08|-103.25|
|154618822706|          51|        The Pizzeria|                  41484|     AT|        Vienna|48.213| 16.413|
|163208757312|          65|       Chef's Corner|                  96638|     GB|        London|51.495| -0.191|
| 68719476763|          28|    The Spicy Pickle|                  77517|     US|      Grayling|44.657|-84.744|
|

In [ ]:
# Read and concatenate data
dataframe_par = read_data(spark, 'data/geo/', 'parquet')

# Show the first few rows of the DataFrame
dataframe_par.show()

Py4JJavaError: An error occurred while calling o89.parquet.
: java.lang.ClassCastException: class java.util.ArrayList cannot be cast to class java.lang.String (java.util.ArrayList and java.lang.String are in module java.base of loader 'bootstrap')
	at org.apache.spark.sql.internal.SessionState.$anonfun$newHadoopConfWithOptions$1(SessionState.scala:110)
	at org.apache.spark.sql.internal.SessionState.$anonfun$newHadoopConfWithOptions$1$adapted(SessionState.scala:110)
	at scala.collection.immutable.Map$Map1.foreach(Map.scala:193)
	at org.apache.spark.sql.internal.SessionState.newHadoopConfWithOptions(SessionState.scala:110)
	at org.apache.spark.sql.execution.datasources.DataSource.newHadoopConfiguration(DataSource.scala:116)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:365)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:229)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$2(DataFrameReader.scala:211)

In [24]:
import os

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        for file in files:
            if file.endswith('.snappy.parquet'):
                print(os.path.join(root, file))

list_files('data/geo/')


data/geo/august\day=01\part-00037-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet
data/geo/august\day=01\part-00038-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet
data/geo/august\day=01\part-00215-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet
data/geo/august\day=02\part-00010-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet
data/geo/august\day=02\part-00011-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet
data/geo/august\day=02\part-00206-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet
data/geo/august\day=03\part-00085-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet
data/geo/august\day=03\part-00086-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet
data/geo/august\day=03\part-00204-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet
data/geo/august\day=04\part-00127-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet
data/geo/august\day=04\part-00128-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet

In [40]:
df = spark.read.parquet('data/geo/august\day=01\part-00037-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet')

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\jksls\AppData\Local\Temp\ipykernel_8368\18468970.py:1: SyntaxWarning: invalid escape sequence '\d'
  df = spark.read.parquet('data/geo/august\day=01\part-00037-44bd3411-fbe4-4e16-b667-7ec0fc3ad489.c000.snappy.parquet')


In [41]:
df.show()

+--------+--------+----------+----------+----------+
|     lng|     lat|avg_tmpr_f|avg_tmpr_c| wthr_date|
+--------+--------+----------+----------+----------+
|-27.6141|-59.7956|       9.8|     -12.3|2017-08-01|
|-27.4095|-59.7956|       9.9|     -12.3|2017-08-01|
| -27.205|-59.7956|       9.9|     -12.3|2017-08-01|
|-27.0004|-59.7956|      10.0|     -12.2|2017-08-01|
|-26.7959|-59.7956|      10.0|     -12.2|2017-08-01|
|-27.8186|-59.5911|       9.8|     -12.3|2017-08-01|
|-27.6141|-59.5911|      10.0|     -12.2|2017-08-01|
|-27.4095|-59.5911|      10.1|     -12.2|2017-08-01|
| -27.205|-59.5911|      10.2|     -12.1|2017-08-01|
|-27.0004|-59.5911|      10.3|     -12.1|2017-08-01|
|-26.7959|-59.5911|      10.4|     -12.0|2017-08-01|
|-27.8186|-59.3867|       9.9|     -12.3|2017-08-01|
|-27.6141|-59.3867|      10.1|     -12.2|2017-08-01|
|-27.4095|-59.3867|      10.3|     -12.1|2017-08-01|
| -27.205|-59.3867|      10.5|     -11.9|2017-08-01|
|-27.0004|-59.3867|      10.7|     -11.8|2017-

In [47]:
api_key = '16a2553fc34b4e64b6e4da5c5183d521'

In [48]:
def fetch_coordinates(api_key, address):
    url = f"https://api.opencagedata.com/geocode/v1/json?q={address}&key={api_key}"
    response = requests.get(url)
    data = response.json()
    return data['results'][0]['geometry']['lat'], data['results'][0]['geometry']['lng']


In [54]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit
import pygeohash as pgh

# Initialize Spark Session
spark = SparkSession.builder.appName("Geolocation and Join Task").getOrCreate()

# Assuming df_restaurant and df_weather are your DataFrames
# Define a function to generate a geohash
def geohash(lat, lng):
    if lat is not None and lng is not None:
        return pgh.encode(lat, lng, precision=4)
    return None

# Register UDF for geohash generation
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

geohash_udf = udf(geohash, StringType())

In [55]:
df = df.withColumn("geohash", geohash_udf(col("lat"), col("lng")))
df

DataFrame[lng: double, lat: double, avg_tmpr_f: double, avg_tmpr_c: double, wthr_date: string, geohash: string]

In [56]:
dataframe = dataframe.withColumn("geohash", geohash_udf(col("lat"), col("lng")))

In [57]:
df_joined = dataframe.join(df, "geohash", "left")

In [60]:
print(df_joined.count())

Py4JJavaError: An error occurred while calling o265.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 24.0 failed 1 times, most recent failure: Lost task 0.0 in stage 24.0 (TID 40) (192.168.0.18 executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:99)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(Interrup

In [59]:
spark.sparkContext.setLogLevel("DEBUG")


In [61]:
df_joined.printSchema()

root
 |-- geohash: string (nullable = true)
 |-- id: long (nullable = true)
 |-- franchise_id: integer (nullable = true)
 |-- franchise_name: string (nullable = true)
 |-- restaurant_franchise_id: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- lat: double (nullable = true)
 |-- avg_tmpr_f: double (nullable = true)
 |-- avg_tmpr_c: double (nullable = true)
 |-- wthr_date: string (nullable = true)



In [58]:
df_joined.show()

Py4JJavaError: An error occurred while calling o265.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 23.0 failed 1 times, most recent failure: Lost task 0.0 in stage 23.0 (TID 39) (192.168.0.18 executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:99)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(Int